# New Wrapper Class

The main wrapper class around a HDF5 file in this package is `H5File` which uses the wrapper class `H5Group` for `h5py.Group` and `H5Dataset` for wrapping `h5py.Dataset`

In [1]:
from h5rdmtoolbox.h5wrapper.h5file import H5File, H5Group, H5Dataset
from h5rdmtoolbox.h5wrapper.h5base import H5BaseDataset, H5Base, H5BaseGroup
import h5py

In [2]:
class MyDataset(H5BaseDataset):
    def __repr__(self):
        return f'{self.name}\n{self.attrs}'
    
    @property
    def is_2d(self):
        """returns whether dataset is two-dimensional or not"""
        return self.ndim == 2

Next we create a group class with a special method, hat returns all datasets of that group. Also the `create_dataset` method is overwritten. Take care to return `MyDataset` at the end of the method, otherwise dataset class of the parent class is taken.

In [3]:
class MyGroup(H5BaseGroup):
    
    def get_all_datasets(self):
        """returns all datasets of this group"""
        return [k for k in self if isinstance(self[k], h5py.Dataset)]
    
    def create_group(self, *args, **kwargs):
        return __class__(super().create_group(*args, **kwargs).id)
    
    # def create_dataset(self, *args, **kwargs):
    #     return __class__(super().create_dataset(*args, **kwargs).id)
    
    def create_dataset(self, name, user, *args, **kwargs):
        ds = super().create_dataset(name, *args, **kwargs)
        ds.attrs.modify('user', user)
        return self._h5ds(ds.id)

The main file wrapper inherites from `H5File` ("root" parent was `h5py.File`) and the new group class. Next, we have set the group and dataset class again, since some methods in the file wrapper class will need that information when returning instances of those classes (e.g. dataset or group creation). Finally we define a new method which sets the user name to the root attributes:

In [4]:
class MyWrapper(H5Base, MyGroup):
    
    def set_user(self, user_name):
        self.attrs.modify('user', user_name)

register the dataset and group class in all classes. This is needed, so all return objects are of the newly defined types

In [5]:
MyGroup._h5ds = MyDataset
MyGroup._h5grp = MyGroup

MyDataset._h5ds = MyDataset
MyDataset._h5grp = MyGroup

In [6]:
h5 = MyWrapper()

C:\Users\da4323\AppData\Local\h5rdmtoolbox\h5rdmtoolbox\tmp\tmp1\tmp0.hdf


In [7]:
type(h5)

__main__.MyWrapper

In [8]:
g = h5.create_group('grp', overwrite=True)

In [9]:
type(g)

__main__.MyGroup

In [10]:
gg = g.create_group('grp', overwrite=True)

In [11]:
type(gg)

__main__.MyGroup

In [12]:
ds = gg.create_dataset('hello', user='test user', shape=(2,3), overwrite=True)

In [13]:
ds

/grp/grp/hello
user  test user

In [14]:
type(ds)

__main__.MyDataset

In [15]:
h5.datasets

[]